In [1]:
import trimesh
def pytorch3d_to_trimesh(p3d_mesh):
    # Get vertices and faces from the PyTorch3D mesh
    vertices = p3d_mesh.verts_packed().detach().cpu().numpy()
    faces = p3d_mesh.faces_packed().detach().cpu().numpy()

    # Create a Trimesh object
    return trimesh.Trimesh(vertices=vertices, faces=faces)

In [ ]:
# If you want to have meshes with different number of vertices:




In [ ]:
def equality_constraints(self, xs: torch.Tensor, y: torch.Tensor):
    """
    Enforces volume constraint
    Assumes same number of vertices in each projected mesh currently

    Args:
        xs (tensor): vertices of original mesh, B x V x 3
        y (tensor): vertices of projected mesh, B x V x 3
    """
    n_batches = len(xs)
    verts = y.view(n_batches,-1,3) # (B, V, 3)

    faces = self.src.faces_padded().view(n_batches, -1, 3).detach()  # (B, max F_i, 3)
    _, n_faces, _ = faces.size()
    batch_indices = torch.arange(n_batches)[:, None, None].expand(-1, n_faces, 3).detach() # detached

    face_vertices = verts[batch_indices, faces]  # (B, F, 3, 3)
    # Calculate volume
    v0, v1, v2 = face_vertices[:, :, 0, :], face_vertices[:, :, 1, :], face_vertices[:, :, 2, :]
    cross = torch.cross(v1, v2, dim=-1)  # (B, F, 3)
    face_volumes = torch.einsum('bfi,bfi->bf', v0, cross) / 6.0  # (B, F)

    volumes = torch.sum(face_volumes, dim=1).abs() # (B,)
    return volumes  # (B,)


In [ ]:
def get_signed_tet_volume(face_vertices: torch.Tensor) -> torch.Tensor:
    """
    Compute signed tetrahedron volumes for a batch of faces.

    Args:
        face_vertices (torch.Tensor): Tensor of shape (F, 3, 3), where
                                      F is the number of faces, and each face
                                      consists of 3 vertices in 3D.

    Returns:
        torch.Tensor: A tensor of shape (F,) containing signed volumes.
    """
    v0, v1, v2 = face_vertices[:, 0, :], face_vertices[:, 1, :], face_vertices[:, 2, :]
    
    # Compute determinant of the 3x3 matrix [v0, v1, v2]
    volumes = torch.det(torch.stack([v0, v1, v2], dim=-1)) / 6.0  # Shape: (F,)

    return volumes

def get_volume_batch(meshes: Meshes):
    verts_packed = meshes.verts_packed()  # (sum(V_i), 3)
    faces_packed = meshes.faces_packed()  # (sum(F_i), 3)
    mesh_to_face = meshes.mesh_to_faces_packed_first_idx()  # Index of first face per mesh
    n_meshes = len(meshes)
    volumes = torch.zeros(n_meshes, device=verts_packed.device)

    for i in range(n_meshes):
        start = mesh_to_face[i]
        end = start + meshes.num_faces_per_mesh()[i]
        face_vertices = verts_packed[faces_packed[start:end]]  # (F, 3, 3)
        volumes[i] = get_signed_tet_volume(face_vertices).sum()  # Sum over all faces

    return volumes.abs()  # Returns a tensor of shape (num_meshes,)

def least_squares(u0, target):
    """
    u0 are vertices
    """
    if not torch.is_tensor(u0):
        u0 = torch.tensor(u0)
    if not torch.is_tensor(target):
        target = torch.tensor(target)

    res = torch.square(u0 - target).sum()
    print("lstsq", res)
    return res.double()

def least_squares_grad(u0, target):
    if torch.is_tensor(u0):
        u0 = u0.detach().clone()
    else:
        u0 = torch.tensor(u0)
        
    if torch.is_tensor(target):
        target = target.detach().clone()
    else:
        target = torch.tensor(target)
        
    # Ensure that u0 requires gradients
    u0.requires_grad = True
    
    with torch.enable_grad():
        res = torch.square(u0 - target).sum()

    # Compute the gradient
    obj_grad = torch.autograd.grad(res, u0)[0]
    print("obj_grad", obj_grad)
    return obj_grad.double()

def get_volume(u, faces, init_vol):
    if not torch.is_tensor(u):
        u = torch.tensor(u)
    if not torch.is_tensor(faces):
        faces = torch.tensor(faces)
    if not torch.is_tensor(init_vol):
        init_vol = torch.tensor(init_vol)
    
    vertices = u.view(-1,3)
    face_vertices = vertices[faces]  # (F, 3, 3)
    volume = get_signed_tet_volume(face_vertices).sum()
    res = volume.abs() - init_vol
    print("vol", res)
    return res.double()

def get_volume_grad(u, faces, init_vol):
    if not torch.is_tensor(u):
        u = torch.tensor(u, dtype=torch.float64, requires_grad=True)
    else:
        u = u.clone().detach().requires_grad_(True)
    
    if not torch.is_tensor(faces):
        faces = torch.tensor(faces, dtype=torch.long)
    
    if not torch.is_tensor(init_vol):
        init_vol = torch.tensor(init_vol, dtype=torch.float64)
    
    with torch.enable_grad():
        vertices = u.view(-1, 3)
        face_vertices = vertices[faces]  # (F, 3, 3)
        volume = get_signed_tet_volume(face_vertices).sum()
        res = volume.abs() - init_vol
    
    volume_grad = torch.autograd.grad(res, u, retain_graph=True)[0]
    print("volume grad", volume_grad)
    
    return volume_grad.double()

def project(meshes: Meshes, targets: Meshes):
    n_batches = len(meshes)
    n_vtxs = len(meshes[0].verts_packed())
    results = torch.zeros(n_batches, n_vtxs, 3, dtype=torch.double)
    losses = torch.zeros(n_batches, 1, dtype=torch.double)
    for batch_number, mesh in enumerate(meshes):
        init_vol = get_volume_batch(mesh).double().detach().cpu().numpy()
        print("batch number", batch_number)
        vertices = mesh.verts_packed().flatten().detach().numpy().astype(np.float64)
        faces = mesh.faces_packed().detach().numpy().astype(np.int64)
        target_vtx = targets[batch_number].verts_packed().flatten().detach().numpy().astype(np.float64)
        eq_const = {
            'type': 'eq',
            'fun' : lambda u: get_volume(u, faces, init_vol).cpu().numpy().astype(np.float64),
            'jac' : lambda u: get_volume_grad(u, faces, init_vol).cpu().numpy().astype(np.float64)
        }
        print("starting optimisation")
        res = opt.minimize(
            lambda u: least_squares(u, target_vtx).detach().cpu().numpy().astype(np.float64),
            vertices, 
            jac=lambda u: least_squares_grad(u, target_vtx).cpu().numpy().astype(np.float64),
            method='SLSQP', 
            constraints=[eq_const],
            options={'ftol': 1e-4, 'disp': True, 'maxiter': 100}
        )   
        print("finished")
        if not res.success:
            print("FIT failed:", res.message)
        results[batch_number] = torch.tensor(res.x, dtype=torch.double, requires_grad=True).view(-1,3)
        losses[batch_number] = torch.tensor(res.fun, dtype=torch.double, requires_grad=False)
    return results, losses
